# Preprocessing of country code

raw data of country codes don't have proper indices, so some preprocessing is required

In [3]:
import pandas as pd
import polars as pl

In [4]:
filepath = "data/official_country_names.csv"
url = "https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv"
try:
    official_country_names_df = pd.read_csv(url)
except Exception as e: # NOTE: fallback to locally stored file if link is dead
    official_country_names_df = pd.read_csv(filepath)
official_country_names_df = pl.from_pandas(official_country_names_df)
official_country_names_df.write_csv(filepath)
official_country_names = official_country_names_df.select("official_name_en").to_dict()["official_name_en"].drop_nulls()
type(official_country_names)
",".join(official_country_names.to_list())
official_country_names = official_country_names.to_list()

In [5]:
lf = pl.scan_csv("data/prf_uchazeci.csv")
pdf = lf.with_columns(pl.col("STAT_OBCANSTVI")).collect()
dataset_countrynames = pdf.select("STAT_OBCANSTVI").unique().to_dict()["STAT_OBCANSTVI"]
",".join(list(dataset_countrynames))

'Libanonská republika,Republika Kazachstán,Ukrajina,Bulharská republika,Íránská islámská republika,Chorvatská republika,Indonéská republika,Ostatní,Namibijská republika,Egyptská arabská republika,Korejská republika,Republika Uzbekistán,Arménská republika,Slovenská republika,Moldavská republika,Maďarsko,Jemenská republika,Ruská federace,Vietnamská socialistická republika,Kyrgyzská republika,Česká republika,Peruánská republika,Stát Izrael,Spolková republika Německo,Ázerbájdžánská republika,Afghánská islámská republika,Běloruská republika'

In [6]:
mapping = {
    'Stát Izrael': 'Israel',
    'Vietnamská socialistická republika': 'Viet Nam',
    'Kyrgyzská republika': 'Kyrgyzstan',
    'Moldavská republika': 'Republic of Moldova',
    'Arménská republika': 'Armenia',
    'Republika Uzbekistán': 'Uzbekistan',
    'Namibijská republika': 'Namibia',
    'Íránská islámská republika': 'Iran (Islamic Republic of)',
    'Spolková republika Německo': 'Germany',
    'Bulharská republika': 'Bulgaria',
    'Ruská federace': 'Russian Federation',
    'Ukrajina': 'Ukraine',
    'Indonéská republika': 'Indonesia',
    'Peruánská republika': 'Peru',
    'Jemenská republika': 'Yemen',
    'Ázerbájdžánská republika': 'Azerbaijan',
    'Chorvatská republika': 'Croatia',
    'Republika Kazachstán': 'Kazakhstan',
    'Afghánská islámská republika': 'Afghanistan',
    'Česká republika': 'Czechia',
    'Korejská republika': 'Republic of Korea',
    'Maďarsko': 'Hungary',
    'Libanonská republika': 'Lebanon',
    'Ostatní': None,
    'Běloruská republika': 'Belarus',
    'Egyptská arabská republika': 'Egypt',
    'Slovenská republika': 'Slovakia',
}
for k, v in mapping.items():
    assert k in dataset_countrynames, f"{k} not in dataset_countrynames"
    assert v is None or v in official_country_names, f"{v} not in official_country_names"
mapping = pl.from_dict({"country_name": list(mapping.keys()), "official_name_en": list(mapping.values())})
# Print the dictionary to confirm the mapping
mapping

country_name,official_name_en
str,str
"""Stát Izrael""","""Israel"""
"""Vietnamská soc…","""Viet Nam"""
"""Kyrgyzská repu…","""Kyrgyzstan"""
"""Moldavská repu…","""Republic of Mo…"
"""Arménská repub…","""Armenia"""
…,…
"""Libanonská rep…","""Lebanon"""
"""Ostatní""",null
"""Běloruská repu…","""Belarus"""


In [7]:
country_codes_df = official_country_names_df.join(mapping, on="official_name_en", how="left")
country_codes_df = country_codes_df.select(["official_name_en", "country_name", "ISO3166-1-Alpha-3"])
country_codes_df = country_codes_df.filter(pl.col("country_name").is_not_null())
country_codes_df

official_name_en,country_name,ISO3166-1-Alpha-3
str,str,str
"""Afghanistan""","""Afghánská islá…","""AFG"""
"""Armenia""","""Arménská repub…","""ARM"""
"""Azerbaijan""","""Ázerbájdžánská…","""AZE"""
"""Belarus""","""Běloruská repu…","""BLR"""
"""Bulgaria""","""Bulharská repu…","""BGR"""
…,…,…
"""Slovakia""","""Slovenská repu…","""SVK"""
"""Ukraine""","""Ukrajina""","""UKR"""
"""Uzbekistan""","""Republika Uzbe…","""UZB"""


In [8]:
country_codes_df.write_csv("data/country_codes.csv")